In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
crsp_data = pd.read_excel("Data/CRSP_fund.xlsx")
#crsp_data

In [3]:
crsp_data.columns

Index(['Frequency: A - annual; Q - quarterly; M - monthly; QM - quarterly and monthly; AQM - annual, quarterly, and monthly',
       'Fund Identifier', 'Date', 'CUSIP (8-digit)', 'Fund Name', 'ticker',
       'Fund CUSIP', 'Management Company Name', 'Management Company Number',
       'Portfolio Manager Name', 'index_fund_flag', 'Date of latest NAV data',
       'Dead Fund Indicator', 'Portfolio Identifier'],
      dtype='object')

In [4]:
crsp_data2 = crsp_data[['Date','Fund Identifier','CUSIP (8-digit)','Fund Name','ticker','Fund CUSIP','Management Company Name','Management Company Number', 'index_fund_flag']]

In [5]:
index_flag_funds = crsp_data2[crsp_data2['index_fund_flag'].isin(['B', 'D', 'E'])]
#index_flag_funds

In [6]:
unique_fund = index_flag_funds['Fund Name'].unique()
with open('unique_fund.txt', 'w') as f:
    for i in unique_fund:
        f.write(f"{i}\n")

In [7]:
lseg_data = pd.read_csv("Data/LSEG_fund.csv")
#lseg_data

In [8]:
lseg_data2 = lseg_data[lseg_data['(shrout1) Shares Outstanding in Millions, as of FDATE'].isna() == False]

In [9]:
unique_funds2 = lseg_data2['(fundname) Fund Name (The name of the mutual abbreviated to fit'].unique()
with open('unique_fund2.txt', 'w') as f:
    for i in unique_funds2:
        f.write(f"{i}\n")

In [10]:
index_flag_funds['Date'] = pd.to_datetime(index_flag_funds['Date'])
lseg_data2['(fdate) File Date'] = pd.to_datetime(lseg_data2['(fdate) File Date'])

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_49521/2091330282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_flag_funds['Date'] = pd.to_datetime(index_flag_funds['Date'])
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_49521/2091330282.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lseg_data2['(fdate) File Date'] = pd.to_datetime(lseg_data2['(fdate) File Date'])
